In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
from IPython.display import display, HTML
import seaborn as sns
import random
import matplotlib.pyplot as plt
import plotly.express as px
import pandas_profiling as pp
from scipy import stats
from scipy.stats import norm
%matplotlib inline
import h2o
from h2o.automl import H2OAutoML

In [5]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('data2/landmarks.csv')
df.columns = [f'col{i}' for i in range(1405)]
df.astype({'col1404':'string'})

Y = df.iloc[:]['col1404']
X = df.drop(columns=['col1404'])
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.15)

In [6]:
ytrain.unique()

array(['surprise', 'neutral', 'fear', 'sad', 'happy', 'contempt', 'anger',
       'disgust'], dtype=object)

In [7]:
h2o.init(
    nthreads=8,     # number of threads when launching a new H2O server
    max_mem_size=8  # in gigabytes
)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /home/vorkov/Workspace/EDA/venv/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp0m8xz18_
  JVM stdout: /tmp/tmp0m8xz18_/h2o_vorkov_started_from_python.out
  JVM stderr: /tmp/tmp0m8xz18_/h2o_vorkov_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_vorkov_rauygq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [8]:
# train.groupby("col1404")["col1404"].count()

In [10]:
train = xtrain
train["col1404"] = ytrain

test = xtest
test["col1404"] = ytest

train_h2o_frame = h2o.H2OFrame(train)
test_h2o_frame = h2o.H2OFrame(test)

x = train_h2o_frame.columns
y = "col1404"
x.remove(y)

aml = H2OAutoML(max_models=8, seed=1, max_runtime_secs=60*60*6, verbosity='info')
aml.train(x=x, y=y, training_frame=train_h2o_frame)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
02:09:39.93: Project: AutoML_2_20231015_20939
02:09:39.93: 5-fold cross-validation will be used.
02:09:39.95: Setting stopping tolerance adaptively based on the training frame: 0.005870886454241044
02:09:39.95: Build control seed: 1
02:09:39.96: training frame: Frame key: AutoML_2_20231015_20939_training_Key_Frame__upload_b852fb71589a6532db795da2f2c5413c.hex    cols: 1405    rows: 29013  chunks: 163    size: 341541227  checksum: -1927708119952564942
02:09:39.96: validation frame: NULL
02:09:39.96: leaderboard frame: NULL
02:09:39.96: blending frame: NULL
02:09:39.96: response column: col1404
02:09:39.96: fold column: null
02:09:39.96: weights column: null
02:09:39.96: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_sear

key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/8
# GBM base models (used / total),4/4
# XGBoost base models (used / total),2/2
# DRF base models (used / total),1/1
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [11]:
aml.distribution

'AUTO'

In [12]:
lb = aml.leaderboard
lb.head(rows=15)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_AllModels_1_AutoML_2_20231015_20939,0.44287,0.89405,0.557507,0.310814
StackedEnsemble_BestOfFamily_1_AutoML_2_20231015_20939,0.445139,0.895831,0.557976,0.311337
GBM_1_AutoML_2_20231015_20939,0.480373,0.955216,0.574188,0.329691
GBM_2_AutoML_2_20231015_20939,0.484976,0.966591,0.57722,0.333183
GBM_3_AutoML_2_20231015_20939,0.48541,0.973295,0.577678,0.333711
XGBoost_1_AutoML_2_20231015_20939,0.488782,1.01193,0.572551,0.327815
GBM_4_AutoML_2_20231015_20939,0.490534,0.991836,0.579007,0.335249
XGBoost_2_AutoML_2_20231015_20939,0.499245,1.02122,0.575854,0.331608
DRF_1_AutoML_2_20231015_20939,0.506707,1.1209,0.59835,0.358022
GLM_1_AutoML_2_20231015_20939,0.509196,1.15889,0.623979,0.38935


In [13]:
test_h2o_frame = test_h2o_frame.drop('col1404')
pred = aml.predict(test_h2o_frame)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [14]:
pred


predict,anger,contempt,disgust,fear,happy,neutral,sad,surprise
neutral,0.000216962,0.0102829,0.00182967,0.000470446,0.0185155,0.93849,0.000275527,0.0299193
disgust,0.204454,0.255072,0.32774,0.053731,0.00168815,0.00653065,0.0818238,0.0689604
neutral,0.000152214,0.0238451,0.00262281,0.000341004,0.425899,0.537482,0.000191289,0.00946637
surprise,0.0992753,0.036846,0.220778,0.182208,0.00278967,0.0243212,0.159305,0.274477
anger,0.825588,0.0400935,0.0551992,0.0270812,0.000170985,0.00159558,0.0447421,0.00552954
disgust,0.317212,0.0781606,0.397671,0.0307651,0.000777959,0.0102348,0.151681,0.0134977
happy,0.00018197,0.00691034,0.0110082,0.00021045,0.866103,0.113448,0.000142809,0.00199499
sad,0.278088,0.0815212,0.0798427,0.165915,0.000506742,0.0058628,0.353705,0.0345585
happy,5.64487e-06,0.000872714,0.000343684,3.3093e-05,0.99712,0.00150537,1.30378e-05,0.000106238
surprise,0.0252043,0.00328589,0.022719,0.318844,0.000232485,0.00185395,0.0326722,0.595188


In [15]:
best_model = aml.leader
# aml.score(test_h2o_frame, test_h2o_frame.MEDV)


In [16]:
h2o.save_model(model=best_model, path="./")

'/home/vorkov/Workspace/EDA/learning/StackedEnsemble_AllModels_1_AutoML_2_20231015_20939'

In [17]:
h2o.load_model('/home/vorkov/Workspace/Emotion-Decetion-Service/learning/GLM_1_AutoML_1_20230909_00442')

H2OResponseError: Server error water.exceptions.H2OIllegalArgumentException:
  Error: Illegal argument: dir of function: importModel: water.api.FSIOException: FS IO Failure: 
 accessed path : file:/home/vorkov/Workspace/Emotion-Decetion-Service/learning/GLM_1_AutoML_1_20230909_00442 msg: File not found
  Request: POST /99/Models.bin/
    data: {'dir': '/home/vorkov/Workspace/Emotion-Decetion-Service/learning/GLM_1_AutoML_1_20230909_00442'}


In [18]:
import numpy as np
from sklearn. metrics import f1_score

pred = aml.predict(test_h2o_frame)
pd=h2o.as_list(pred) 
f1_score(test['col1404'], pd["predict"].tolist(), average="weighted")

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


0.6621331358509505

In [19]:
test['col1404']

15585     neutral
27869        fear
13648     neutral
1832     contempt
17781       anger
           ...   
6522        happy
2382     contempt
14307     neutral
20312     disgust
9335     surprise
Name: col1404, Length: 5120, dtype: object

In [20]:
pd["predict"].tolist()

['neutral',
 'disgust',
 'neutral',
 'surprise',
 'anger',
 'disgust',
 'happy',
 'sad',
 'happy',
 'surprise',
 'surprise',
 'surprise',
 'sad',
 'anger',
 'disgust',
 'surprise',
 'surprise',
 'happy',
 'neutral',
 'disgust',
 'anger',
 'fear',
 'happy',
 'disgust',
 'happy',
 'happy',
 'disgust',
 'anger',
 'happy',
 'disgust',
 'sad',
 'surprise',
 'surprise',
 'neutral',
 'neutral',
 'disgust',
 'happy',
 'surprise',
 'anger',
 'contempt',
 'surprise',
 'happy',
 'surprise',
 'happy',
 'happy',
 'anger',
 'surprise',
 'surprise',
 'neutral',
 'neutral',
 'anger',
 'surprise',
 'sad',
 'contempt',
 'anger',
 'anger',
 'contempt',
 'happy',
 'neutral',
 'contempt',
 'disgust',
 'happy',
 'contempt',
 'sad',
 'surprise',
 'disgust',
 'surprise',
 'happy',
 'contempt',
 'happy',
 'happy',
 'neutral',
 'happy',
 'sad',
 'sad',
 'surprise',
 'neutral',
 'happy',
 'neutral',
 'surprise',
 'happy',
 'happy',
 'neutral',
 'happy',
 'anger',
 'fear',
 'surprise',
 'surprise',
 'sad',
 'neut

In [21]:
f1_score(test['col1404'], pd["predict"].tolist(), average="weighted")

0.6621331358509505